In [1]:
import tensorflow as tf
import string
import requests
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13353310592296749415
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 263847501125401840
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [3]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [4]:
!cat /proc/meminfo

MemTotal:       13298580 kB
MemFree:        10271820 kB
MemAvailable:   11810624 kB
Buffers:           68264 kB
Cached:          1454668 kB
SwapCached:            0 kB
Active:          1293032 kB
Inactive:        1382912 kB
Active(anon):     927748 kB
Inactive(anon):    10692 kB
Active(file):     365284 kB
Inactive(file):  1372220 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               488 kB
Writeback:             0 kB
AnonPages:       1153364 kB
Mapped:           634528 kB
Shmem:             11456 kB
KReclaimable:      85500 kB
Slab:             140092 kB
SReclaimable:      85500 kB
SUnreclaim:        54592 kB
KernelStack:        5220 kB
PageTables:        11524 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6649288 kB
Committed_AS:    3889376 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       51276 kB
VmallocChunk:          0 kB
Percpu:          

In [5]:
response.text
data = response.text.split('\n')

In [6]:
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [7]:
len(data)

124457

In [8]:
data[253:][0]   # from here correct data was there (actual text starts here)

'  From fairest creatures we desire increase,'

In [9]:
data = data[253:]

In [10]:
len(data)

124204

In [11]:
data = ' '.join(data)

In [12]:
def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('','',string.punctuation)
    tokens = [(w.translate(table)) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [13]:
tokens = clean_text(data)

In [14]:
print(tokens[:20])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time']


In [15]:
len(tokens)

898199

In [16]:
len(set(tokens))

27956

In [17]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 200000:
        break

In [18]:
print(len(lines))

199951


In [19]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [20]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

In [21]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences

[[47,
  1408,
  1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
  754,
  7,
  3806,
  99,
  2430,
  466,
  31,
  307],
 [1408,
  1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
  754,
  7,
  3806,
  99,
  2430,
  466,
  31,
  307,
  31],
 [1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
 

In [23]:
sequences = np.array(sequences)
sequences

array([[  47, 1408, 1264, ...,  466,   31,  307],
       [1408, 1264,   37, ...,   31,  307,   31],
       [1264,   37,  451, ...,  307,   31, 1582],
       ...,
       [  33,   80, 5197, ...,  215,   44,   30],
       [  80, 5197,  103, ...,   44,   30, 1332],
       [5197,  103, 1846, ...,   30, 1332,    2]])

In [24]:
x , y = sequences[:, :-1], sequences[:, -1]

In [25]:
print(x[0])
print(y[0])

[   47  1408  1264    37   451  1406     9  2766  1158  1213   171   132
   269    20    24     1  4782    87    30    98  4781    18   715  1263
   171   211    18   829    20    27  3807     4   214   121  1212   153
 13004    31  2765  1847    16 13003 13002   754     7  3806    99  2430
   466    31]
307


In [26]:
x[0].shape

(50,)

In [27]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'to': 4,
 'of': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'that': 9,
 'in': 10,
 'is': 11,
 'not': 12,
 'it': 13,
 'for': 14,
 'me': 15,
 'with': 16,
 'he': 17,
 'his': 18,
 'your': 19,
 'but': 20,
 'be': 21,
 'this': 22,
 'have': 23,
 'as': 24,
 'him': 25,
 'so': 26,
 'thou': 27,
 'will': 28,
 'what': 29,
 'by': 30,
 'thy': 31,
 'no': 32,
 'are': 33,
 'all': 34,
 'her': 35,
 'do': 36,
 'we': 37,
 'if': 38,
 'our': 39,
 'or': 40,
 'shall': 41,
 'thee': 42,
 'which': 43,
 'on': 44,
 'lord': 45,
 'o': 46,
 'from': 47,
 'good': 48,
 'more': 49,
 'sir': 50,
 'was': 51,
 'they': 52,
 'well': 53,
 'at': 54,
 'would': 55,
 'when': 56,
 'now': 57,
 'come': 58,
 'love': 59,
 'th': 60,
 'than': 61,
 'am': 62,
 'then': 63,
 'she': 64,
 'their': 65,
 'them': 66,
 'how': 67,
 'enter': 68,
 'let': 69,
 'did': 70,
 'ill': 71,
 'hath': 72,
 'one': 73,
 'us': 74,
 'know': 75,
 'first': 76,
 'make': 77,
 'had': 78,
 'like': 79,
 'here': 80,
 'upon': 81,
 'there': 82,
 'man': 83,


In [28]:
len(tokenizer.word_index)

13008

In [29]:
vocab_size = len(tokenizer.word_index) + 1

In [30]:
len(set(tokens))

27956

In [31]:
y = to_categorical(y, num_classes = vocab_size)

In [32]:
seq_length = x.shape[1]

In [33]:
seq_length

50

In [34]:
# Building Model

model = Sequential()

model.add(Embedding(input_dim = vocab_size, output_dim = 50, input_length = seq_length))

model.add(LSTM(units = 100, return_sequences = True))

model.add(LSTM(units = 100))

model.add(Dense(units = 100, activation = 'relu'))

model.add(Dense(units = vocab_size, activation = 'softmax'))

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            650450    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 13009)             1313909   
                                                                 
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x,y, batch_size = 256, epochs = 100)

In [ ]:
lines[12345]

In [ ]:
seed_text = lines[12345]

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating = 'pre')
        
         #y_predict = model.predict(encoded)
        y_predict = model.predict(encoded) 
        classes_x=np.argmax(y_predict,axis=1)

        predicted_word = ''
        for word , index in tokenizer.word_index.items():
            if index == classes_x:
                predicted_word = word
                break
            seed_text = seed_text + ' ' + predicted_word
            text.append(predicted_word)
        return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 10)